# How to add memory to chatbots

resource: https://python.langchain.com/docs/how_to/chatbots_memory/

Documentations:
* [Persistence](https://langchain-ai.github.io/langgraph/concepts/persistence/)
* [Memory](https://langchain-ai.github.io/langgraph/concepts/memory/)
* [Add Persistance](https://langchain-ai.github.io/langgraph/how-tos/persistence/#use-with-functional-api)

In [1]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

In [2]:
MODEL = "cogito:8b"
DEEP_THINKING_INSTRUCTION = "Enable deep thinking subroutine.\n\n"

In [3]:
model = ChatOllama(model=MODEL, extract_reasoning=True)

## Message passing

Pass the previous messages to the model

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a helpful assistant. Answer all questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


In [5]:
chain = prompt | model

ai_msg = chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="Translate from English to French: I love programming."
            ),
            AIMessage(content="J'adore la programmation."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

ai_msg

AIMessage(content='I translated "I love programming" into French, and wrote that my response was "J\'adore la programmation." (This means "I love programming" in French.)', additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-06-07T14:57:32.2786154Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2517837500, 'load_duration': 19952000, 'prompt_eval_count': 65, 'prompt_eval_duration': 303526900, 'eval_count': 37, 'eval_duration': 2193327300, 'model_name': 'cogito:8b'}, id='run-c18a529b-f45a-497a-8596-5758714cce36-0', usage_metadata={'input_tokens': 65, 'output_tokens': 37, 'total_tokens': 102})

In [6]:
print(ai_msg.content)

I translated "I love programming" into French, and wrote that my response was "J'adore la programmation." (This means "I love programming" in French.)


## Automatic history management

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [5]:
workflow = StateGraph(state_schema=MessagesState)

In [6]:
# Define the function that calls the model
def call_model(state: MessagesState):
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + state["messages"]
    response = model.invoke(messages)
    return {"messages": response}



In [7]:
# Define the node and edge
workflow.add_node(node="model", action=call_model)
workflow.add_edge(START, "model")

In [8]:
# Add simple in_memory chackpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [9]:
app.invoke(
    {"messages": [HumanMessage(content="Translate to French: I love programming.")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content='Translate to French: I love programming.', additional_kwargs={}, response_metadata={}, id='bfcc8d6d-0cf0-4aa4-a964-73b473aed5cd'),
  AIMessage(content='In French, this is:\n"J\'aime le programmation."', additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-06-08T08:58:58.6670582Z', 'done': True, 'done_reason': 'stop', 'total_duration': 16881174800, 'load_duration': 15059491200, 'prompt_eval_count': 39, 'prompt_eval_duration': 947245200, 'eval_count': 15, 'eval_duration': 868717400, 'model_name': 'cogito:8b'}, id='run-1d21f8c7-c87a-4c01-ab0c-03ab2be7a21b-0', usage_metadata={'input_tokens': 39, 'output_tokens': 15, 'total_tokens': 54})]}

In [10]:
app.invoke(
    {"messages": [HumanMessage(content="What did I just ask you?")]},
    config={"configurable": {"thread_id": "1"}},
)

{'messages': [HumanMessage(content='Translate to French: I love programming.', additional_kwargs={}, response_metadata={}, id='bfcc8d6d-0cf0-4aa4-a964-73b473aed5cd'),
  AIMessage(content='In French, this is:\n"J\'aime le programmation."', additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-06-08T08:58:58.6670582Z', 'done': True, 'done_reason': 'stop', 'total_duration': 16881174800, 'load_duration': 15059491200, 'prompt_eval_count': 39, 'prompt_eval_duration': 947245200, 'eval_count': 15, 'eval_duration': 868717400, 'model_name': 'cogito:8b'}, id='run-1d21f8c7-c87a-4c01-ab0c-03ab2be7a21b-0', usage_metadata={'input_tokens': 39, 'output_tokens': 15, 'total_tokens': 54}),
  HumanMessage(content='What did I just ask you?', additional_kwargs={}, response_metadata={}, id='041b5c06-fb0a-4b40-a4b3-44fc328a2575'),
  AIMessage(content='You asked me to translate "I love programming" into French, which resulted in "J\'aime le programmation."', additional_kwargs={}, re

In [ ]:
# for msg in app.get_state_history({"thread_id": "1"}):
#     print(msg)

## Modifying chat history

#### Trimming messages

In [11]:
demo_ephemeral_chat_history = [
    HumanMessage(content="Hey there! I'm Nemo."),
    AIMessage(content="Hello!"),
    HumanMessage(content="How are you today?"),
    AIMessage(content="Fine thanks!"),
]

app.invoke(
    {
        "messages": demo_ephemeral_chat_history
        + [HumanMessage(content="What's my name?")]
    },
    config={"configurable": {"thread_id": "2"}},
)

{'messages': [HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}, id='c289e52f-6ba2-45c5-b204-3b9dcbff7635'),
  AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='a7b6c010-d081-42a6-99e1-832f6c6fb0a3'),
  HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}, id='7913fa44-6e64-4ca6-9e82-4db67c36249a'),
  AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}, id='7ade540e-4708-4e94-a236-5f27cb96eca7'),
  HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}, id='07d86906-ad5a-4873-bd60-ba9461863a3d'),
  AIMessage(content='Your name is Nemo.', additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-06-08T08:59:16.9412752Z', 'done': True, 'done_reason': 'stop', 'total_duration': 720225900, 'load_duration': 22194200, 'prompt_eval_count': 74, 'prompt_eval_duration': 346990100, 'eval_count': 7, 'eval_duration': 349987100,

In [12]:
from langchain_core.messages import trim_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [13]:
# Define trimmer
# count each message as 1 "token" (token_counter=len) and keep only the last two messages
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

In [14]:
workflow = StateGraph(state_schema=MessagesState)

In [15]:
# Define the function that calls the model
def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    system_prompt = (
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability."
    )
    messages = [SystemMessage(content=system_prompt)] + trimmed_messages
    response = model.invoke(messages)
    return {"messages": response}


In [16]:
# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

In [ ]:
# Add simple in-memory checkpointer
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [18]:
app.invoke(
    {
        "messages": demo_ephemeral_chat_history
        + [HumanMessage(content="What is my name?")]
    },
    config={"configurable": {"thread_id": "3"}},
)

{'messages': [HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}, id='c289e52f-6ba2-45c5-b204-3b9dcbff7635'),
  AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='a7b6c010-d081-42a6-99e1-832f6c6fb0a3'),
  HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}, id='7913fa44-6e64-4ca6-9e82-4db67c36249a'),
  AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}, id='7ade540e-4708-4e94-a236-5f27cb96eca7'),
  HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}, id='df9c4736-bfb0-415c-af9d-a20b75c5bcc8'),
  AIMessage(content="I don't actually know what your name is - you haven't told me yet! Feel free to share if you'd like, but I'm happy to help without needing it.", additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-06-08T09:06:44.2018149Z', 'done': True, 'done_reason': 'stop', 'total_duration': 11329193800, 'loa

#### Summary memory

In [36]:
demo_ephemeral_chat_history = [
    HumanMessage(content="Hey there! I'm Nemo."),
    AIMessage(content="Hello!"),
    HumanMessage(content="How are you today?"),
    AIMessage(content="Fine thanks!"),
    HumanMessage(content="I come from Mars"),
    AIMessage(content="Oh, really?!"),
]

In [37]:
from langchain_core.messages import HumanMessage, RemoveMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [45]:
workflow = StateGraph(state_schema=MessagesState)

In [46]:
# Define the function that calls the model
def call_model(state: MessagesState):
    system_prompt = (
        # "Enable deep thinking subroutine.\n\n"
        "You are a helpful assistant. "
        "Answer all questions to the best of your ability. "
        "The provided chat history includes a summary of the earlier conversation."
    )
    system_message = SystemMessage(content=system_prompt)
    message_history = state["messages"][:-1]  # exclude the most recent user input
    print("History\n", state["messages"])
    print()
    # Summarize the messages if the chat history reaches a certain size
    if len(message_history) >= 4:
        last_human_message = state["messages"][-1]
        print("Last human message:", last_human_message)
        print()
        # Invoke the model to generate conversation summary
        summary_prompt = (
            # "Enable deep thinking subroutine.\n\n"
            "Distill the above chat messages into a single summary message. "
            "Include as many specific details as you can."
        )
        summary_message = model.invoke(
            message_history + [HumanMessage(content=summary_prompt)]
        )

        # Delete messages that we no longerwant to show up
        delete_messages = [RemoveMessage(id=m.id) for idx, m in enumerate(state["messages"]) if idx < 4]
        print("Delete\n", delete_messages)
        print()
        # Re-add user message
        human_message = HumanMessage(content=last_human_message.content)
        # Call the model with summary and response
        response = model.invoke([system_message, summary_message, human_message])
        message_updates = [summary_message, human_message, response] + delete_messages
    else:
        message_updates = model.invoke([system_message] + state["messages"])

    return {"messages": message_updates}

In [47]:
# Define the node and edge
workflow.add_node("model", call_model)
workflow.add_edge(START, "model")

In [48]:
# Add simple in-memory checkpointer
memory =  MemorySaver()
app = workflow.compile(checkpointer=memory)

In [49]:
list(app.get_state_history(config={"configurable": {"thread_id": "4"}}))

[]

In [50]:
resp = app.invoke(
    {
        "messages": demo_ephemeral_chat_history
        + [HumanMessage("What did I say my name was?")]
    },
    config={"configurable": {"thread_id": "4"}},
)

History
 [HumanMessage(content="Hey there! I'm Nemo.", additional_kwargs={}, response_metadata={}, id='fbe0d2ab-bd0e-4d91-b3b0-49e1f3a7d03a'), AIMessage(content='Hello!', additional_kwargs={}, response_metadata={}, id='5da53c29-4da7-4c5c-bd06-ce0b82c4b55e'), HumanMessage(content='How are you today?', additional_kwargs={}, response_metadata={}, id='f010cea9-6187-4216-a156-36183c9ffeaf'), AIMessage(content='Fine thanks!', additional_kwargs={}, response_metadata={}, id='4aa78b48-4b04-467f-b7e5-1315c0bb3b2f'), HumanMessage(content='I come from Mars', additional_kwargs={}, response_metadata={}, id='ce957387-a2f5-44ff-a7fc-f7dc7318e304'), AIMessage(content='Oh, really?!', additional_kwargs={}, response_metadata={}, id='916e6f43-70d6-45da-8fe9-0f99ae4a5358'), HumanMessage(content='What did I say my name was?', additional_kwargs={}, response_metadata={}, id='32f99c66-e1b5-4c6b-99b8-ac8ce49be6f4')]

Last human message: content='What did I say my name was?' additional_kwargs={} response_metadata

In [51]:
resp

{'messages': [HumanMessage(content='I come from Mars', additional_kwargs={}, response_metadata={}, id='ce957387-a2f5-44ff-a7fc-f7dc7318e304'),
  AIMessage(content='Oh, really?!', additional_kwargs={}, response_metadata={}, id='916e6f43-70d6-45da-8fe9-0f99ae4a5358'),
  HumanMessage(content='What did I say my name was?', additional_kwargs={}, response_metadata={}, id='32f99c66-e1b5-4c6b-99b8-ac8ce49be6f4'),
  AIMessage(content="Here's a summary of our conversation:\n\n- Initial greeting where I introduced myself as an AI assistant\n- You introduced yourself as Nemo\n- We exchanged basic pleasantries (asking how each other was)\n- You mentioned coming from Mars, which is quite an interesting claim!", additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-06-15T06:01:15.6334189Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3846021900, 'load_duration': 26253800, 'prompt_eval_count': 87, 'prompt_eval_duration': 349677300, 'eval_count': 57, 'eval_durati

In [53]:
resp = app.invoke(
    {
        "messages": resp["messages"]
        + [HumanMessage("I am whqt you, humans, call an alien")]
    },
    config={"configurable": {"thread_id": "4"}},
)

History
 [HumanMessage(content='I come from Mars', additional_kwargs={}, response_metadata={}, id='ce957387-a2f5-44ff-a7fc-f7dc7318e304'), AIMessage(content='Oh, really?!', additional_kwargs={}, response_metadata={}, id='916e6f43-70d6-45da-8fe9-0f99ae4a5358'), HumanMessage(content='What did I say my name was?', additional_kwargs={}, response_metadata={}, id='32f99c66-e1b5-4c6b-99b8-ac8ce49be6f4'), AIMessage(content="Here's a summary of our conversation:\n\n- Initial greeting where I introduced myself as an AI assistant\n- You introduced yourself as Nemo\n- We exchanged basic pleasantries (asking how each other was)\n- You mentioned coming from Mars, which is quite an interesting claim!", additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-06-15T06:01:15.6334189Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3846021900, 'load_duration': 26253800, 'prompt_eval_count': 87, 'prompt_eval_duration': 349677300, 'eval_count': 57, 'eval_duration': 34690

In [54]:
resp

{'messages': [HumanMessage(content='What did I say my name was?', additional_kwargs={}, response_metadata={}, id='e38be286-9ae0-4380-b0fc-642cf0dc42d3'),
  AIMessage(content='You said your name was "Nemo".', additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-06-15T06:01:16.4160164Z', 'done': True, 'done_reason': 'stop', 'total_duration': 779290300, 'load_duration': 18768600, 'prompt_eval_count': 112, 'prompt_eval_duration': 217810800, 'eval_count': 10, 'eval_duration': 542180000, 'model_name': 'cogito:8b'}, id='run-36042ea9-0d08-4855-b719-3514efa37004-0', usage_metadata={'input_tokens': 112, 'output_tokens': 10, 'total_tokens': 122}),
  HumanMessage(content='I am whqt you, humans, call an alien', additional_kwargs={}, response_metadata={}, id='720f7161-b396-47a1-9ed6-7bb6e5797b23'),
  AIMessage(content="In our brief conversation, you introduced yourself as Nemo and mentioned that you come from Mars. I responded to both statements but didn't ask for clari

In [55]:
resp = app.invoke(
    {
        "messages": resp["messages"]
        + [HumanMessage("I don't know what to say. The way I am is normal for me. What would you like to know?")]
    },
    config={"configurable": {"thread_id": "4"}},
)

History
 [HumanMessage(content='What did I say my name was?', additional_kwargs={}, response_metadata={}, id='e38be286-9ae0-4380-b0fc-642cf0dc42d3'), AIMessage(content='You said your name was "Nemo".', additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-06-15T06:01:16.4160164Z', 'done': True, 'done_reason': 'stop', 'total_duration': 779290300, 'load_duration': 18768600, 'prompt_eval_count': 112, 'prompt_eval_duration': 217810800, 'eval_count': 10, 'eval_duration': 542180000, 'model_name': 'cogito:8b'}, id='run-36042ea9-0d08-4855-b719-3514efa37004-0', usage_metadata={'input_tokens': 112, 'output_tokens': 10, 'total_tokens': 122}), HumanMessage(content='I am whqt you, humans, call an alien', additional_kwargs={}, response_metadata={}, id='720f7161-b396-47a1-9ed6-7bb6e5797b23'), AIMessage(content="In our brief conversation, you introduced yourself as Nemo and mentioned that you come from Mars. I responded to both statements but didn't ask for clarification a

In [56]:
resp

{'messages': [HumanMessage(content='I am whqt you, humans, call an alien', additional_kwargs={}, response_metadata={}, id='b807c6b0-0c40-4c96-be4a-551b7b835072'),
  AIMessage(content='Yes, I understand now - you\'re confirming that you are indeed what humans would refer to as an "alien," which in this context means someone who isn\'t human, perhaps coming from another planet or having a different origin. Is there anything specific about your nature as an alien that you\'d like to discuss?', additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-06-15T06:02:31.6512795Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4232266100, 'load_duration': 18334300, 'prompt_eval_count': 112, 'prompt_eval_duration': 215684000, 'eval_count': 63, 'eval_duration': 3997739300, 'model_name': 'cogito:8b'}, id='run-3cdfed43-d2c0-43d3-81a0-de4ef762d262-0', usage_metadata={'input_tokens': 112, 'output_tokens': 63, 'total_tokens': 175}),
  HumanMessage(content="I don't kno

In [57]:
resp = app.invoke(
    {
        "messages": resp["messages"]
        + [HumanMessage("You are quite egocentric. I have been told that humans often act this way.")]
    },
    config={"configurable": {"thread_id": "4"}},
)

History
 [HumanMessage(content='I am whqt you, humans, call an alien', additional_kwargs={}, response_metadata={}, id='b807c6b0-0c40-4c96-be4a-551b7b835072'), AIMessage(content='Yes, I understand now - you\'re confirming that you are indeed what humans would refer to as an "alien," which in this context means someone who isn\'t human, perhaps coming from another planet or having a different origin. Is there anything specific about your nature as an alien that you\'d like to discuss?', additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-06-15T06:02:31.6512795Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4232266100, 'load_duration': 18334300, 'prompt_eval_count': 112, 'prompt_eval_duration': 215684000, 'eval_count': 63, 'eval_duration': 3997739300, 'model_name': 'cogito:8b'}, id='run-3cdfed43-d2c0-43d3-81a0-de4ef762d262-0', usage_metadata={'input_tokens': 112, 'output_tokens': 63, 'total_tokens': 175}), HumanMessage(content="I don't know what t

In [58]:
resp

{'messages': [HumanMessage(content="I don't know what to say. The way I am is normal for me. What would you like to know?", additional_kwargs={}, response_metadata={}, id='98a881da-8454-44ec-a4f2-b4264a229888'),
  AIMessage(content="It's completely natural not to know how to react or respond when faced with an unexpected situation. Your reaction of feeling unsure makes sense because this encounter is different from typical interactions. If you're comfortable, we could start by exploring what aspects of my identity interest you most - whether it's about the nature of my existence on Mars, my experiences here on Earth, or something else entirely. What would you like to know more about?", additional_kwargs={}, response_metadata={'model': 'cogito:8b', 'created_at': '2025-06-15T06:06:54.931407Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5603610200, 'load_duration': 18864400, 'prompt_eval_count': 135, 'prompt_eval_duration': 255376000, 'eval_count': 87, 'eval_duration': 5328317